In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# Load data into a pandas DataFrame
data = pd.read_csv('combined_data.csv')
data = data.dropna()

# Separate the feature (textual data) and target (binary labels) columns
X = data['text']
y = data['isHate']

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Vectorize the textual data using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [6]:
# Oversample the texts class
smote = SMOTE(random_state=42)
X_train_tfidf_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

In [7]:
# Train Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf_resampled, y_train_resampled)

LogisticRegression()

In [8]:
# Train Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf_resampled, y_train_resampled)

MultinomialNB()

In [9]:
# Train Support Vector Machine
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train_tfidf_resampled, y_train_resampled)

SVC(kernel='linear')

In [10]:
X_train_tfidf_resampled = X_train_tfidf_resampled.toarray()

In [11]:
# Simple Neural Network architecture
nn_model = Sequential()
nn_model.add(Dense(64, input_shape=(X_train_tfidf_resampled.shape[1],), activation='relu'))
nn_model.add(Dropout(0.5)) 
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dropout(0.5)) 
nn_model.add(Dense(1, activation='sigmoid'))

In [12]:
# Compile the model
nn_model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
es = EarlyStopping(patience=50, restore_best_weights=True, monitor="loss")

In [14]:
# Train the model
nn_model.fit(X_train_tfidf_resampled, y_train_resampled, epochs=10, batch_size=32, verbose=1, callbacks=[es])

Epoch 1/10
1197/1197 [==============================] - 51s 18ms/step - loss: 0.4137 - accuracy: 0.8067
Epoch 2/10
1197/1197 [==============================] - 24s 20ms/step - loss: 0.0870 - accuracy: 0.9728
Epoch 3/10
1197/1197 [==============================] - 13s 11ms/step - loss: 0.0483 - accuracy: 0.9859
Epoch 4/10
1197/1197 [==============================] - 11s 9ms/step - loss: 0.0354 - accuracy: 0.9900
Epoch 5/10
1197/1197 [==============================] - 11s 9ms/step - loss: 0.0288 - accuracy: 0.9918
Epoch 6/10
1197/1197 [==============================] - 11s 9ms/step - loss: 0.0225 - accuracy: 0.9939
Epoch 7/10
1197/1197 [==============================] - 11s 10ms/step - loss: 0.0196 - accuracy: 0.9948
Epoch 8/10
1197/1197 [==============================] - 11s 9ms/step - loss: 0.0189 - accuracy: 0.9946
Epoch 9/10
1197/1197 [==============================] - 11s 9ms/step - loss: 0.0135 - accuracy: 0.9961
Epoch 10/10
1197/1197 [==============================] - 12s 10ms/ste

In [15]:
# Make predictions on the testing set
lr_pred = lr_model.predict(X_test_tfidf)
nb_pred = nb_model.predict(X_test_tfidf)
svm_pred = svm_model.predict(X_test_tfidf)
nn_pred = nn_model.predict_classes(X_test_tfidf)

/Users/student/opt/anaconda3/lib/python3.8/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [16]:
# Print the classification report and accuracy score for each model
print("Tf-Idf Results")
print()

print('Logistic Regression:')
print(classification_report(y_test, lr_pred))
print('Accuracy score:', accuracy_score(y_test, lr_pred))
print()


print('Naive Bayes:')
print(classification_report(y_test, nb_pred))
print('Accuracy score:', accuracy_score(y_test, nb_pred))
print()


print('Support Vector Machine:')
print(classification_report(y_test, svm_pred))
print('Accuracy score:', accuracy_score(y_test, svm_pred))
print()

print('Neural Network:')
print(classification_report(y_test, nn_pred))
print('Accuracy score:', accuracy_score(y_test, nn_pred))

Tf-Idf Results

Logistic Regression:
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      4763
           1       0.32      0.61      0.42       390

    accuracy                           0.87      5153
   macro avg       0.64      0.75      0.68      5153
weighted avg       0.92      0.87      0.89      5153

Accuracy score: 0.8734717640209587

Naive Bayes:
              precision    recall  f1-score   support

           0       0.97      0.85      0.91      4763
           1       0.27      0.65      0.38       390

    accuracy                           0.84      5153
   macro avg       0.62      0.75      0.64      5153
weighted avg       0.91      0.84      0.87      5153

Accuracy score: 0.8379584707937124

Support Vector Machine:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      4763
           1       0.30      0.52      0.38       390

    accuracy                        